# (1) 데이터 검색 및 내용 작성하기

In [ ]:
!pip install 'crewai[tools]'

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # openai 키 입력
os.environ["SERPER_API_KEY"] = os.getenv('SERPER_API_KEY') # SERPER 키 입력

In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# 구글 검색 API를 활용해 정보를 검색하는 기능을 제공
search_tool = SerperDevTool()

searching=Agent(
    role='Searching',
    goal='{topic}에 대한 적절한 내용을 찾아주세요',
    verbose=True,
    memory=True,
    backstory=("당신은 주제에 대한 검색에 최적화되어 있습니다."),
    tools=[search_tool],
    allow_delegation=True # 다른 에이전트에 작업을 위임할 수 있게 허용
)   # searching 에이전트

answer=Agent(
    role='Writing',
    goal='{topic}에 대한 내용을 설명하세요',
    verbose=True,
    memory=True,
    backstory=("간결하면서도 명확하게 작성하세요."),
    tools=[search_tool],
    allow_delegation=False 
)   # answer 에이전트

In [ ]:
from crewai import Task # 크루AI에서 특정 작업을 정의하고 실행

search_task = Task(
    description=(
        "{topic}에서 주목할 만한 키워드를 파악하세요. 장점과 단점을 식별하고 전반적인 관점으로 분석하세요."
    ),
    expected_output='{topic}에 검색 결과를 3문단 분량으로 정리하세요.',
    tools=[search_tool],
    agent=searching,
)   # 첫 번째 작업, searching 에이전트를 사용해 {topic}에 대한 검색 결과를 작성

answer_task = Task(
    description=(
        "{topic}에 대한 통찰력 있는 기사를 작성하세요. 최신 동향과 그것의 파급력을 중점적으로 다루세요."
    ),
    expected_output='{topic}에 관한 4문단짜리 기사로, 마크다운 형식으로 작성하세요',
    tools=[search_tool],
    agent=answer,
    async_execution=False,
    output_file='/content/new-blog-post.md' # 결과 지정한 파일에 저장
)   # 두 번째 작업, answer 에이전트를사용하여 {topic}에 대한 기사를 작성


In [ ]:
result = crew.kickoff(
    inputs={'topic': 'AI 에이전트가 AI 산업에 미치는 영향은?'}
)   # crew 객체에 정의된 작업과 에이전트를 실행
print(result)

# (2) 유튜브 채널에서 데이터 검색

In [ ]:
!pip install 'crewai[tools]'

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # openai 키 입력

In [ ]:
!pip install yt_dlp==2023.11.14 youtube-transcript-api==0.6.1

In [ ]:
from crewai_tools import YoutubeChannelSearchTool

# 특정 유튜브 채널에서 데이터 검색하기
yt_tool = YoutubeChannelSearchTool(
    youtube_channel_handle='@AI-km1yn'
)

In [ ]:
from crewai import Agent, LLM

searching = Agent(
    role='YouTube 비디오에서 전문가 수준의 콘텐츠를 연구',
    goal='{topic}와 관련된 유튜브 비디오의 모든 콘텐츠를 검색하세요.',
    verbose=True,
    memory=True,
    backstory=(
        'AI에 관한 유튜브 비디오를 이해하고 파악하는데 전문가.'
        '머신러닝(Machine Learning), 딥러닝(Deep Learning), 생성형 AI(Generative AI)에 관한 유튜브 비디오를 이해하고 분석하며, 관련 콘텐츠를 제공하는 전문가.'
    ),
    tool = [yt_tool],
    allow_delegation=True
)   # searching 에이전트, 유튜브에서 도메인 전문가 수준의 콘텐츠를 검색하고 분석

answer = Agent(
    role='기사 작성자',
    goal='유튜브 비디오 주제: {topic}에 대한 체계적이고 유익한 기사를 작성하세요.',
    verbose=True,
    memory=True,
    backstory=(
        '복잡한 기술적인 내용을 명확히 이해하고 체계적으로 답변을 작성하세요.'
    ),
    tool = [yt_tool],
    allow_delegation=False
)   # answer 에이전트, searching 에이전트가 수집한 정보를 바탕으로 기사를 작성

In [ ]:
from crewai import Task

search_task = Task(
    description=(
        "YouTube 비디오 {topic}을 검색하세요."
        "채널에서 비디오에 대한 전문적이고 자세한 콘텐츠나 정보를 가져오세요."
    ),
    expected_output='{topic}을 주제로 검색한 내용을 4단락으로 정리하세요.', # 작업의 예상 결과
    tools=[yt_tool],
    agent=searching
)   # search_task는 주어진 {topic}과 관련된 유튜브 데이터를 수집하고 분석

answer_task = Task(
    description=(
        "YouTube 채널의 비디오에서 {topic}에 대한 기사를 작성하세요."
    ),
    expected_output='YouTube 채널의 내용을 요약하고, YouTube 채널 비디오를 바탕으로 {topic}에 대한 자세하고 체계적인 기사를 작성하세요.',
    tools=[yt_tool],
    agent=answer,
    async_execution=False,
    output_file='/content/new-article.md'
)   # answer_task는 search_task에서 수집된 정보를 바탕으로 {topic}에 대한 기사를 작성

In [ ]:
from crewai import Crew, Process

crew = Crew( # crew 객체 생성
    agents=[searching, answer],
    tasks=[search_task, answer_task],
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)

result = crew.kickoff(
    inputs={'topic': 'AI Agent가 무엇인가요?'}
)   # 작업 실행
print(result)